In [1]:
import os
import sys

import pandas as pd
import numpy as np
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
from Platform.database.mysql import MysqlAPI
from datetime import datetime, timedelta

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn import metrics

from Research.feature.ft import FeatureAnalysis
from Research.utils import namespace as namespace
from Research.utils import normalization as norm

config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
feature_path=r"/home/ShareFolder/feature_platform"

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [5]:
start_time = "2019-01-01"
end_time = "2021-06-01"
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=feature_path)
alpha_list = list(FT.features_in_path.keys())[:3]
FT = FeatureAnalysis(configs, feature_path=feature_path)
a = FT.load_feature_from_file(alpha_list, start_time, end_time, universe='Investable',
                          timedelta=None, transformer=norm.standard_scale)[0]
a

Loading Feature...: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1752/1752 [00:29<00:00, 60.04it/s]


COV_open_high_10  COV_high_close_10  \
timestamp           ticker                                        
2019-01-17 09:30:00 600652         46.530039         -11.015658   
                    600653         27.579905          45.782908   
                    600651         18.310132          21.893643   
                    600601         13.597187          26.312377   
                    000651          2.200963           4.881275   
...                                      ...                ...   
2021-05-19 09:30:00 000677         -0.053591          -0.053336   
                    300730         -0.053598          -0.053284   
                    300683         -0.053669          -0.052797   
                    300940         -0.053745          -0.052926   
                    300391         -0.053866          -0.052915   

                            COV_high_low_10  
timestamp           ticker                   
2019-01-17 09:30:00 600652        11.782406  
                    600653        47.433174  
                    600651        24.260409  
                    600601        21.046899  
                    000651         3.826755  
...                                     ...  
2021-05-19 09:30:00 000677        -0.051470  
                    300730        -0.051442  
                    300683        -0.051505  
                    300940        -0.051446  
                    300391        -0.051596  

[2108534 rows x 3 columns]

In [6]:
FT.feature_data

COV_open_high_10  COV_high_close_10  \
timestamp           ticker                                        
2019-01-17 09:30:00 600652         46.530039         -11.015658   
                    600653         27.579905          45.782908   
                    600651         18.310132          21.893643   
                    600601         13.597187          26.312377   
                    000651          2.200963           4.881275   
...                                      ...                ...   
2021-05-19 09:30:00 000677         -0.053591          -0.053336   
                    300730         -0.053598          -0.053284   
                    300683         -0.053669          -0.052797   
                    300940         -0.053745          -0.052926   
                    300391         -0.053866          -0.052915   

                            COV_high_low_10  
timestamp           ticker                   
2019-01-17 09:30:00 600652        11.782406  
                    600653        47.433174  
                    600651        24.260409  
                    600601        21.046899  
                    000651         3.826755  
...                                     ...  
2021-05-19 09:30:00 000677        -0.051470  
                    300730        -0.051442  
                    300683        -0.051505  
                    300940        -0.051446  
                    300391        -0.051596  

[2108534 rows x 3 columns]

In [4]:
a[0]

COV_open_high_10  COV_high_close_10  \
timestamp           ticker                                        
2019-01-02 09:30:00 600652         58.750201          58.706183   
                    600651          1.319094           2.413985   
                    600653          0.973202           1.126982   
                    600612          0.750403           0.965750   
                    600601          0.743283           0.994322   
...                                      ...                ...   
2021-05-31 09:30:00 300479         -0.040684          -0.037358   
                    300162         -0.040706          -0.037299   
                    300257         -0.040730          -0.035732   
                    300196         -0.040863          -0.037468   
                    688027         -0.040965          -0.035575   

                            COV_high_low_10  
timestamp           ticker                   
2019-01-02 09:30:00 600652        58.704452  
                    600651         2.301378  
                    600653         1.243730  
                    600612         0.938054  
                    600601         1.158616  
...                                     ...  
2021-05-31 09:30:00 300479        -0.038001  
                    300162        -0.038031  
                    300257        -0.037849  
                    300196        -0.038189  
                    688027        -0.037234  

[2108534 rows x 3 columns]

In [40]:
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=feature_path)
alpha_list = list(FT.features_in_path.keys())[:10]

def test_get_correlation_within_features(alpha_list, start_time, end_time,bench_mark = 0.9):
    FT = FeatureAnalysis(configs, feature_path=feature_path)
    FT.load_feature_from_file(alpha_list, start_time, end_time, universe='Investable',
                              timedelta=None, transformer=norm.standard_scale)
    final_alpha_list = []
    Corr={}
    for i in alpha_list:
        if final_alpha_list == []:
            final_alpha_list.append(i)
            continue
        corr_table = FT.get_correlation_within_features(i, start_time=start_time , end_time=end_time,others=final_alpha_list)
        if corr_table["correlation"].max() <= abs(bench_mark * 100):
            final_alpha_list.append(i)
        else:
            print("delete [%s] because of %d [%s]" % (i,corr_table["correlation"].max(),corr_table["correlation"].idxmax()))
    FT.feature_data = FT.feature_data[final_alpha_list]
    return FT.feature_data , final_alpha_list

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [41]:
a , b = test_get_correlation_within_features(alpha_list,"2019-01-01","2021-06-01")

Calculating correlations...: 100%|█████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 631.25it/s]


delete [COV_high_close_10] because of 97.70157941258859 [COV_open_high_10]


Calculating correlations...: 100%|█████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 431.46it/s]


delete [COV_high_low_10] because of 98.77902510831888 [COV_open_high_10]


Calculating correlations...: 100%|█████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 504.23it/s]


...

delete [COV_low_volume_10] because of 91.06589394273156 [COV_low_turnover_10]


Calculating correlations...: 100%|█████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 630.67it/s]


delete [COV_low_vwap_10] because of 98.19513956552414 [COV_open_high_10]


Calculating correlations...: 100%|█████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 583.30it/s]


delete [COV_low_close_10] because of 97.67906974336621 [COV_open_high_10]


In [20]:
a

['COV_open_high_10',
 'COV_open_free_turnover_10',
 'COV_low_turnover_10',
 'COV_low_pct_chg_10']

In [29]:
start_time = "2019-01-01"
end_time = "2021-06-01"

FT = FeatureAnalysis(configs, feature_path=feature_path)
FT.load_feature_from_file(alpha_list, start_time, end_time, universe='Investable',
                          timedelta=None, transformer=norm.standard_scale)
Corr={}
for i in alpha_list:
    corr_table = FT.get_correlation_within_features(i, start_time=start_time , end_time=end_time )
    Corr[i]=corr_table

Process ForkPoolWorker-728:
Process ForkPoolWorker-723:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Exception ignored in: <function _releaseLock at 0x7f745e859680>
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/p

  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py",

In [50]:
"%d" % corr_table["correlation"].max()
corr_table["correlation"].max()
# Corr['COV_open_high_10']["correlation"]

99.41899616757397

In [23]:
FT.load_feature_from_file(alpha_list, start_time, end_time, universe='Investable',
                              timedelta=None, transformer=norm.standard_scale)

Loading Feature...: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5840/5840 [00:18<00:00, 320.42it/s]


(                            COV_open_high_10  COV_high_close_10  \
 timestamp           ticker                                        
 2019-01-22 09:30:00 600519         41.267244          40.164386   
                     600653         30.227240          30.653497   
                     000651         18.004749          20.616139   
                     600652         13.311333          15.933766   
                     600612         10.994661           8.462858   
 ...                                      ...                ...   
 2021-05-28 09:30:00 300302         -0.043847          -0.040235   
                     002285         -0.043854          -0.039772   
                     600415         -0.043870          -0.040409   
                     600458         -0.043881          -0.040203   
                     600340         -0.043921          -0.037952   
 
                             COV_high_low_10  COV_open_free_turnover_10  \
 timestamp           ticker            

In [16]:
a["correlation"].max()

8.885900785789946